In [20]:
from scrawl import *
import random
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse


c = Crawl() # Creating object

# Creating list of proxies
proxy_col = dashboard['proxies']
proxy = proxy_col.find({'country':'China'})
proxy_list = list(map(lambda x:x['ip'],list(proxy)))


def get_proxy():
    
    return {'http': random.choice(proxy_list)}


def download_image(image_url,image_path):

    time.sleep(random.randint(1,3))
    try:
        r = requests.get('https://s.weibo.com/top/summary?cate=realtimehot',proxies=get_proxy(),timeout=120)
        response_code = r.status_code
        if response_code != 200:
            return f'Unable to fetch image {image_url} Error code: {r.status_code}\n'
            # print(f'Unable to fetch image {image_url} Error code: {r.status_code}')
        else:
            r = r.content
            with open(image_path, 'wb') as f:
                f.write(r)
    except:
        return f'Unable to fetch image {image_url} Unknown error\n'
        # print(f'Unable to fetch image {image_url} Unknown error\n')

    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5eb16c2db378671cf746ebb8'  
site = 's_weibo_com_hot'
cl_data = dashboard['core_weibo']

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_username = 0
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_download_image = 0
publish_source = 's.weibo.com'
country = 'China'
language = 'Chinese'
cnt = 0
publish_time = '00:00:00'
pub_date = created_on  # no pattern to find date
# Hence we will concate date(which is in a sentence) in the source content.
no_of_not_working_queries = 0
headers = {
    'authority': 'weibo.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '^\\^',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
    'cookie': 'SINAGLOBAL=764815322341.5566.1622097283265; SUB=2AkMXj8zTf8NxqwJRmP0RzmrjaY1yyg3EieKh0z0IJRMxHRl-yT92qmgntRB6PA_iPI199P4zlRz9zonVc5W23plzUH7V; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9W55o9Nf.NuDNjNQuIS8pJY; _s_tentry=-; Apache=3847225399074.1636.1624690011593; ULV=1624690011604:5:4:4:3847225399074.1636.1624690011593:1624608998989',
}
user_links = ['https://weibo.com/globaltimescn?refer_flag=1001030103_&ssl_rnd=1623643790.2876',
             'https://weibo.com/newsxh?refer_flag=1001030103_',
             'https://weibo.com/u/7547570847',
             'https://weibo.com/hebgqt?refer_flag=1001030103_&is_all=1',
             'https://weibo.com/xhstyb?refer_flag=1001030103_',
             'https://weibo.com/cctvnewsbeijing',
             'https://weibo.com/u/3704995720?refer_flag=1001030103_',
             'https://weibo.com/badnewscn?nick=%E4%B8%8A%E6%B5%B7%E4%B9%90%E6%A2%A6_Bad_News_China',
             'https://weibo.com/chinadailywebsite?ssl_rnd=1623648346.4715',
             'https://weibo.com/chinadailywebsite?ssl_rnd=1623648346.4715&is_all=1',
             'https://weibo.com/weather01?refer_flag=1001030103_',
             'https://weibo.com/cctvnewsbeijing?refer_flag=1001030103_',
             'https://weibo.com/cctvnewsbeijing?refer_flag=1001030103_&ssl_rnd=1623648683.3943&is_all=1',
             'https://weibo.com/baincompany?refer_flag=1001030103_&ssl_rnd=1623649023.9775&is_all=1',
             'https://weibo.com/u/6139776919?refer_flag=1001030103_',
             'https://weibo.com/ttarticle/p/show?id=2309404621567283036299',
             'https://weibo.com/zhongguowang?refer_flag=1001030103_',
             'https://weibo.com/chinadailywebsite?ssl_rnd=1623648346.4715&is_all=1',
             'https://weibo.com/cctv5',
             'https://weibo.com/cgtnenespanol?refer_flag=1001030103_',
             'https://weibo.com/chinanewsv',
             'https://weibo.com/cctvnewsbeijing?refer_flag=1001030103_&is_all=1',
             'https://weibo.com/cctvnewsbeijing?refer_flag=1001030103_&ssl_rnd=1623650973.4547&is_all=1',
             'https://weibo.com/cctvnewsbeijing?refer_flag=1001030103_&ssl_rnd=1623650973.4547&is_all=1',
             'https://weibo.com/gxrb2013?refer_flag=1001030103_&ssl_rnd=1623654240.3591',
             'https://weibo.com/gmrb1949?ssl_rnd=1623654193.1401s',
             'https://weibo.com/beijingdaily',
             'https://weibo.com/thepapernewsapp?nick=%E6%BE%8E%E6%B9%83%E6%96%B0%E9%97%BB',
             'https://s.weibo.com/claim/apply?object_id=1022:231522de583bd8b6ab3e27312f5d2d87d081ea&uid=0&query=%23%E7%8E%AF%E7%90%83%E6%97%B6%E6%8A%A5%23',
             'https://weibo.com/u/6314326330',
             'https://weibo.com/shinebridal?is_all=1']

foot_fall = c.get_foot_fall(publish_source)
for user_link in user_links:
    
    print(user_link)
    logger.info(f'Fetching cat url  {user_link}\n') 
    try:
        user_page = requests.get(user_link,headers = headers ,timeout=120).text
    except:
        print("time error")
    if user_page.startswith('Unable to fetch'):
        logger.info(user_page) # writes error message with error code
        no_of_not_working_queries += 1
        continue   
    #class="WB_feed_detail clearfix"
    for i in user_page.split('WB_feed_detail clearfix')[1:3]:

        # Date and time #<a name=\"2021-06-24 22:57\"
        date_time = c.scrap('<a\s*name=.*?title=.*?"(.*?)"', i) # %Y%m%d%H%M
        date_time  = re.sub(r'\\','',date_time)  # r is used to remove this special symbol (\) from the string
        date_time = c.strip_html(date_time)
        print("date_time::",date_time)
        source_content = c.scrap('<div\s*class=.*?"WB_text W_f14.*?".*?>(.*?)<div.*?class=.*?"WB_expand_media_box', i) + '\n' + date_time
        source_content = c.strip_html(source_content)
        source_content = re.sub(r'\\n','',source_content)  # this is how to subsitute the special symbol using regex,  use prefix (r) with it
        print('source_content',source_content)

        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {user_link}\n')
            skipped_due_to_content += 1
            continue
        # handle duplicates
        content_query = {'content':source_content}
        dic = cl_data.find_one(content_query,{'content': 1}) 
        if dic:
            duplicate_data += 1
            continue

#         # source_headline
        source_headline = source_content
        print("source_headline",source_headline)
        username = c.scrap('nick=(.*?)\&', i)
        print("username",username)
    

#         # skip if username not found
        if not username:
            logger.info(f'Skipping due to username {user_link}\n')
            skipped_due_to_username += 1
            continue  
      # journalist
        journalist = ''
        if not journalist: journalist = 'NA'

        # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
        images_path = []
        
        favourites = c.scrap('click:fav">.*?>(.*?)<', i)
        favourites = c.scrap('(\d+)', favourites)
        
        forward = c.scrap('click:repost.*?>(.*?)<', i)
        forward = c.scrap('(\d+)', forward)
        
        comments = c.scrap('click:comment.*?>(.*?)<', i)
        comments = c.scrap('(\d+)', comments)        
        
        likes = c.scrap('click:like.*?<em>(.*?)</em>', i)
        likes = c.scrap('(\d+)', likes)
        
        # Image
        img_urls = re.findall('<li>\s*<img\s*src="(.*?)"', i, flags=re.S)

        for img_url in img_urls:
            img_name = c.scrap('.*/(.*)', img_url)
            img_path = f'{image_directory}/{img_name}'
            if not img_url.startswith('http'):
                img_url = 'https:' + img_url
            # print(img_url)
            # if image is not downloaded return an error message
            download_message = download_image(img_url, img_path)  
            images_path.append(img_path)
            
            if download_message:
                logger.info(download_message)  # writes error message with error code
                unable_to_download_image += 1
                continue  
           
        # storing the above data in a dictionary
        clientdata = {
                        "client_master": client_id,
                        "articleid": client_id,
                        "medium": "Weibo",
#                         "searchkeyword": [query], 
                        "entityname": [], 
                        "process_flage": "1",
                        "na_flage": "0",
                        "na_reason": "",
                        "qc_by": "",
                        "qc_on": "",
                        "location": "",
                        "spokeperson": "",
                        "quota": "",
                        "overall_topics": "",
                        "person": "",
                        "overall_entites": "",
                        "overall_tonality": "",
                        "overall_wordcount": word_count,
                        "article_subjectivity": "",
                        "article_summary": "",
                        "pub_date": pub_date,
                        "publish_time": publish_time,
                        "harvest_time": harvest_time,  
                        "temp_link": user_link,
                        "publish_source": username,  
                        "programme": "null",
                        "feed_class": "",
                        "publishing_platform": "Weibo",
                        "klout_score": "0",
                        "journalist": "", 
                        "headline": source_content,  
                        "content": source_content,  
                        "language": language,
                        "location_mention": "",
                        "source_link":user_link,
                        "author_contact": "",
                        "author_emailid": "",
                        "author_url": "",
                        "city": "",
                        "state": "",
                        "country": country,
                        "source": username,
                        "foot_fall": "",
                        "created_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        "active": "1",
                        "circulation": "0",
                        'favourites':favourites,
                        'forward': forward,
                        'comments':comments,
                        'likes': likes,
                        'images_path': images_path
        }

#         cl_data.insert_one(clientdata)  
        no_of_data += 1
        

logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
# logger.info(f'Total number of queries: {len(queries)}\n')
logger.info(f'No. of not working queries: {no_of_not_working_queries}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Skipped due to username: {skipped_due_to_username}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Unable to download image: {unable_to_download_image}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

https://weibo.com/globaltimescn?refer_flag=1001030103_&ssl_rnd=1623643790.2876
date_time:: 2021-06-24 22:57
source_content                                                         置顶                                                                                                                        【香港苹果把自己搞成了煽动政治对抗的舆论中心】西方的新闻自由必须与他们的国家利益和公共安全是一致的，却要求苹果日报拥有破坏香港安全和中国国家利益的自由，他们为这两种新闻自由注入了南辕北辙的价值取向。LGlobalTimes的秒拍视频 ​​​​                                            
2021-06-24 22:57
source_headline                                                         置顶                                                                                                                        【香港苹果把自己搞成了煽动政治对抗的舆论中心】西方的新闻自由必须与他们的国家利益和公共安全是一致的，却要求苹果日报拥有破坏香港安全和中国国家利益的自由，他们为这两种新闻自由注入了南辕北辙的价值取向。LGlobalTimes的秒拍视频 ​​​​                                            
2021-06-24 22:57
username 环球时报-英文版
date_time:: 2021-06-30 14:30
source_content                                                                              

https://weibo.com/ttarticle/p/show?id=2309404621567283036299
https://weibo.com/zhongguowang?refer_flag=1001030103_
https://weibo.com/chinadailywebsite?ssl_rnd=1623648346.4715&is_all=1
date_time:: 2021-06-30 09:07
source_content                                                         置顶                                                                                                                        #一场虚拟的特殊面试#【这份工作没有退休时间，全世界却有9000多万人在做】 #中国共产党党员总数为9514.8万名#如果有这样一份工作，它要求：不辞劳苦、不惧危险、不计付出、不问回报；紧急情况24小时随叫随到；一旦入职，没有退休时间…… 你猜有人做吗？#一份9500多万人做却没有退休的工作# ​​​​...展开全文c                                            
2021-06-30 09:07
source_headline                                                         置顶                                                                                                                        #一场虚拟的特殊面试#【这份工作没有退休时间，全世界却有9000多万人在做】 #中国共产党党员总数为9514.8万名#如果有这样一份工作，它要求：不辞劳苦、不惧危险、不计付出、不问回报；紧急情况24小时随叫随到；一旦入职，没有退休时间…… 你猜有人做吗？#一份9500多万人做却没有退休的工作# ​​​​...展开全文c                  

https://weibo.com/thepapernewsapp?nick=%E6%BE%8E%E6%B9%83%E6%96%B0%E9%97%BB
https://s.weibo.com/claim/apply?object_id=1022:231522de583bd8b6ab3e27312f5d2d87d081ea&uid=0&query=%23%E7%8E%AF%E7%90%83%E6%97%B6%E6%8A%A5%23
https://weibo.com/u/6314326330
https://weibo.com/shinebridal?is_all=1


In [16]:
# print(cat_page)

In [60]:
import re
a = 'a\"2021-06-24 22:57\"'
re.search('"(.*?)"',a).group(1)

'2021-06-24 22:57'

In [61]:
a

'a"2021-06-24 22:57"'

'<html><head>\n    <meta http-equiv="Content-type" content="text/html; charset=gb2312">\n    <title>Sina Visitor System</title>\n</head>\n<body>\n<span id="message"></span>\n<script type="text/javascript" src="/js/visitor/mini_original.js?v=20161116"></script>\n<script type="text/javascript">\n    window.use_fp = "1" == "1"; // 是否采集设备指纹。\n    var url = url || {};\n    (function () {\n        this.l = function (u, c) {\n            try {\n                var s = document.createElement("script");\n                s.type = "text/javascript";\n                s[document.all ? "onreadystatechange" : "onload"] = function () {\n\n                    if (document.all && this.readyState != "loaded" && this.readyState != "complete") {\n                        return\n                    }\n                    this[document.all ? "onreadystatechange" : "onload"] = null;\n                    this.parentNode.removeChild(this);\n                    if (c) {\n                        c()\n            

In [33]:
import re
a='''
div class=\"WB_text W_f14\" node-type=\"feed_list_content\" nick-name=\"环球时报-英文版\">
<a ignore=\"ignore\"  href=\"javascript:void(0);\"><span class=\"W_icon_feedpin\"><i class=\"W_icon icon_feedpin_lite\" title=\"微博会员特权\"><\/i>置顶<\/span><\/a>\n
【香港苹果把自己搞成了煽动政治对抗的舆论中心】西方的新闻自由必须与他们的国家利益和公共安全是一致的，却要求苹果日报拥有破坏香港安全和中国国家利益的自由，他们为这两种新闻自由注入了南辕北辙的价值取向。
<a  suda-uatrac
'''

In [44]:
re.search('div\s*class="WB_text W_f14".*?title=(.*?)<a',a,re.S).group(1)

'"微博会员特权"><\\/i>置顶<\\/span><\\/a>\n\n【香港苹果把自己搞成了煽动政治对抗的舆论中心】西方的新闻自由必须与他们的国家利益和公共安全是一致的，却要求苹果日报拥有破坏香港安全和中国国家利益的自由，他们为这两种新闻自由注入了南辕北辙的价值取向。\n'